In [90]:
import pandas as pd

df = pd.read_csv('dataset/in.tsv', sep='\t')
df.columns = ["file_name", "list_keys", "pdf2djvu/djvu2hocr", "tesseract", "textract", "pdf2djvu/djvu2hocr_tesseract"]
df = df.dropna(axis = 0)
df.head()
df.shape

(1682, 6)

In [87]:
import sweetviz as sv

my_report = sv.analyze(df)
# my_report.show_html() # Default arguments will generate to "SWEETVIZ_REPORT.html"

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


In [ ]:
# import spacy

# # Load English tokenizer, tagger, parser and NER
# nlp = spacy.load("en_core_web_sm")

# # Process whole documents
# text = (df["tesseract"][1])
# doc = nlp(text)

# document = [sent.orth_ for sent in doc.sents]



In [91]:
document = df["tesseract"].values.tolist()

In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(document)

true_k = 6
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])

Cluster 0:
 college
 school
 governors
 nthe
 year
 pupils
 000
 financial
 august
 funds
Cluster 1:
 year
 nthe
 charity
 church
 trustees
 funds
 accounts
 financial
 report
 trust
Cluster 2:
 financial
 nthe
 year
 charity
 trustees
 funds
 statements
 company
 31
 ended
Cluster 3:
 000
 nthe
 year
 financial
 company
 31
 march
 statements
 charity
 funds
Cluster 4:
 optional
 charity
 nearest
 nsection
 trustees
 ntar
 funds
 nadditional
 ne
 information
Cluster 5:
 00
 year
 charity
 nthe
 trustees
 accounts
 church
 n0
 club
 2017
